In [1]:
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import matplotlib as mpl
from matplotlib.pyplot import cm

In [3]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [4]:
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
from matplotlib_venn import venn3

## Get GTEx eQTLs

In [5]:
sig_eqtl="/cellar/users/mpagadal/Data/gtex/GTEx_Analysis_v8_eQTL_EUR"
files=[x for x in os.listdir(sig_eqtl) if "pairs" in x]

In [6]:
mapping=pd.read_csv("/cellar/users/mpagadal/Data/gtex/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.lookup_table.txt",delimiter="\t")
mp_b38_b37=dict(zip(mapping.variant_id,mapping.variant_id_b37))

In [7]:
#get snp lst
snps=pd.read_csv("/cellar/users/mpagadal/Data2/projects/germline-immune/snp-tables/immune-microenvironment-nonhla.tsv",delimiter="\t")
snps["assoc"]=snps["snps"]+"_"+snps["file"]
snp_lst=snps["snps"].tolist()+[x.rsplit(":",2)[0]+":"+x.split(":")[3]+":"+x.split(":")[2] for x in snps["snps"].tolist()]

In [8]:
compiled_cis=pd.DataFrame()

for x in files:
    print(x)
    eqtl=pd.read_csv(sig_eqtl+"/"+x,delimiter="\t")
    eqtl["variant_b37"] = eqtl["variant_id"].map(mp_b38_b37)
    eqtl["variant"] = eqtl["variant_b37"].str.split("_").str[0]+":"+eqtl["variant_b37"].str.split("_").str[1]+":"+eqtl["variant_b37"].str.split("_").str[2]+":"+eqtl["variant_b37"].str.split("_").str[3]
    eqtl["cell"]=x.split(".v8")[0]
    eqtl_filt=eqtl[eqtl["variant"].isin(snp_lst)]
    compiled_cis=compiled_cis.append(eqtl_filt)

Spleen.v8.EUR.signif_pairs.txt
Uterus.v8.EUR.signif_pairs.txt
Lung.v8.EUR.signif_pairs.txt
Artery_Coronary.v8.EUR.signif_pairs.txt
Pancreas.v8.EUR.signif_pairs.txt
Brain_Hippocampus.v8.EUR.signif_pairs.txt
Artery_Tibial.v8.EUR.signif_pairs.txt
Brain_Spinal_cord_cervical_c-1.v8.EUR.signif_pairs.txt
Skin_Not_Sun_Exposed_Suprapubic.v8.EUR.signif_pairs.txt
Muscle_Skeletal.v8.EUR.signif_pairs.txt
Colon_Sigmoid.v8.EUR.signif_pairs.txt
Brain_Caudate_basal_ganglia.v8.EUR.signif_pairs.txt
Prostate.v8.EUR.signif_pairs.txt
Vagina.v8.EUR.signif_pairs.txt
Cells_EBV-transformed_lymphocytes.v8.EUR.signif_pairs.txt
Adipose_Subcutaneous.v8.EUR.signif_pairs.txt
Minor_Salivary_Gland.v8.EUR.signif_pairs.txt
Heart_Left_Ventricle.v8.EUR.signif_pairs.txt
Brain_Frontal_Cortex_BA9.v8.EUR.signif_pairs.txt
Brain_Cortex.v8.EUR.signif_pairs.txt
Esophagus_Mucosa.v8.EUR.signif_pairs.txt
Adrenal_Gland.v8.EUR.signif_pairs.txt
Brain_Nucleus_accumbens_basal_ganglia.v8.EUR.signif_pairs.txt
Skin_Sun_Exposed_Lower_leg.v8.E

In [ ]:
compiled_cis["gene_id"]=compiled_cis["phenotype_id"].str.split(".").str[0]

In [ ]:
with open('/cellar/users/mpagadal/Data/resources/tcga/ensembl_map.json', 'r') as f:
    ensembl_dict = json.load(f)

In [ ]:
ensembl_dict={k.split(".")[0]:v for k,v in ensembl_dict.items()}

In [ ]:
compiled_cis["gtex gene"]=compiled_cis["gene_id"].map(ensembl_dict)

In [ ]:
compiled_cis=compiled_cis.rename(columns={"variant":"snps"})

In [ ]:
compiled_cis["assoc"]=compiled_cis["snps"]+"_"+compiled_cis["gtex gene"]

## combine gtex and tcga data frame

In [ ]:
gtex_assoc=pd.merge(snps,compiled_cis[["snps","gtex gene","cell","gene_id"]],on="snps",how="left")

In [ ]:
cis=gtex_assoc[~(gtex_assoc["cis tss"].isnull())]["assoc"].unique()
trans=gtex_assoc[gtex_assoc["cis tss"].isnull()]["assoc"].unique()

In [ ]:
len(trans)

In [ ]:
gtex_cis=gtex_assoc[gtex_assoc["assoc"].isin(cis)]
gtex_cis=gtex_cis[gtex_cis["gtex gene"]==gtex_cis["file"]]["assoc"].unique().tolist()

In [ ]:
len(gtex_cis)

In [ ]:
gtex_trans=gtex_assoc[gtex_assoc["assoc"].isin(trans)]
gtex_trans=gtex_trans[~(gtex_trans["gtex gene"].isnull())]["assoc"].unique().tolist()

In [ ]:
len(gtex_trans)

In [ ]:
f=plt.figure(figsize=(5,5))
plt.bar(["cis","trans"],[len(cis),len(trans)])
plt.bar(["cis","trans"],[len(gtex_cis),len(gtex_trans)],label="gtex eQTL")
plt.ylabel("number of associations")
plt.xlabel("type of association")
plt.legend()
plt.savefig("../plots/gtex.eqtl.pdf")

In [ ]:
len(trans)

## Compare Whole Blood eQTLs and Tissue-type eQTLs

In [ ]:
combos=['Kidney_Cortex.KICH',
 'Kidney_Cortex.KIRC',
 'Kidney_Cortex.KIRP',
 'Testis.TGCT',
 'Brain_Hippocampus.GBM',
 'Skin_Sun_Exposed_Lower_leg.SKCM',
 'Ovary.OV',
 'Colon_Transverse.COAD',
 'Adrenal_Gland.ACC',
 'Pancreas.PAAD',
 'Esophagus_Muscularis.ESCA',
 'Stomach.STAD',
 'Thyroid.THCA',
 'Breast_Mammary_Tissue.BRCA',
 'Minor_Salivary_Gland.HNSC',
 'Brain_Anterior_cingulate_cortex_BA24.LGG',
 'Liver.LIHC',
 'Prostate.PRAD',
 'Uterus.UCEC',
 'Lung.LUAD',
 'Lung.LUSC']
gtex_tissues=[x.split(".")[0] for x in combos]

In [ ]:
overlap=compiled_cis[compiled_cis["cell"].isin(gtex_tissues+["Whole_Blood"])]

In [ ]:
fig, axs = plt.subplots(3, 6, sharey=True, tight_layout=True, figsize=(15, 10))
axli = axs.flatten()

for i,y in enumerate([x for x in set(gtex_tissues)]):
    print(y)
    venn2([set(overlap[overlap["cell"]==y]["assoc"].tolist()),set(overlap[overlap["cell"]=="Whole_Blood"]["assoc"].tolist())],
     set_labels=("",""), set_colors=["green","gray"],ax=axli[i])
    axli[i].title.set_text(y)
plt.show()

In [ ]:
snps=pd.read_csv("/cellar/users/mpagadal/Data/projects/germline-immune/snp-tables/immune-microenvironment-nonhla.tsv",delimiter="\t")

In [ ]:
blood_snps=compiled_cis[(compiled_cis["cell"]=="Whole_Blood")&(compiled_cis["gtex gene"].isin(snps["file"].tolist()))]

In [ ]:
brca_snps=compiled_cis[(compiled_cis["cell"]=="Breast_Mammary_Tissue")&(compiled_cis["gtex gene"].isin(snps["file"].tolist()))]

In [ ]:
set([x for x in blood_snps["gtex gene"] if x not in brca_snps["gtex gene"].tolist()])

In [ ]:
set([x for x in brca_snps["gtex gene"] if x not in blood_snps["gtex gene"].tolist()])

## Get QQ plot of GTEx cis snps

In [ ]:
def compile_gtex(directory,file_name,mapping):
    gtex_files=[x for x in os.listdir(directory)]
    total_gtex=pd.DataFrame()

    for x in gtex_files:
        cis=pd.read_csv(directory+x)
        cis["file"]=x.split(file_name)[0]
        total_gtex=total_gtex.append(cis)
    
    #format variant2 and gene
    total_gtex["variant2"]=total_gtex["variant_b37"].str.split("_").str[0]+":"+total_gtex["variant_b37"].str.split("_").str[1]+":"+total_gtex["variant_b37"].str.split("_").str[3]+":"+total_gtex["variant_b37"].str.split("_").str[2]
    #get gene ids for mapping
    total_gtex["gene_id"]=total_gtex["phenotype_id"].str.split(".").str[0]
    
    total_gtex["ref"]=total_gtex["variant_id"].str.split("_").str[2]
    total_gtex["alt"]=total_gtex["variant_id"].str.split("_").str[3]
    
    total_gtex_melt=pd.melt(total_gtex, id_vars=(["gene_id","slope","slope_se","ref","alt","file","pval_nominal","maf","tss_distance","variant_b37"]),value_vars=(["variant","variant2"]))
    
    total_gtex_melt["gene_name"]=total_gtex_melt["gene_id"].map(mapping)
    
    total_gtex_melt["assoc"]=total_gtex_melt["value"]+"_"+total_gtex_melt["gene_name"]
    
    return(total_gtex_melt)

In [ ]:
gtex=compile_gtex("/cellar/users/mpagadal/Data/gtex/extracted_snps/",".gtex.cis.eur.snps.csv",ensembl_dict)

In [ ]:
df_gtex_cis=gtex[gtex["assoc"].isin(gtex_cis)]

In [ ]:
len(df_gtex_cis["assoc"].unique())

In [ ]:
tpm_cancer=pd.read_csv("/cellar/users/mpagadal/Data/projects/germline-immune/snp-tables/brian-cancer-type-beta-se.tsv",delimiter="\t")
firebrowse_cancer=pd.read_csv("/cellar/users/mpagadal/Data/projects/germline-immune/snp-tables/firebrowse-cancer-type-beta-se.tsv",delimiter="\t")
pancanatlas_cancer=pd.read_csv("/cellar/users/mpagadal/Data/projects/germline-immune/snp-tables/pancanatlas-cancer-type-beta-se.tsv",delimiter="\t")

In [ ]:
compare_beta=pd.merge(df_gtex_cis,tpm_cancer,on="assoc")
compare_beta["combo"]=compare_beta["file"]+"."+compare_beta["file_name"]

In [ ]:
combos=['Kidney_Cortex.KICH',
 'Kidney_Cortex.KIRC',
 'Kidney_Cortex.KIRP',
 'Testis.TGCT',
 'Brain_Hippocampus.GBM',
 'Skin_Sun_Exposed_Lower_leg.SKCM',
 'Ovary.OV',
 'Colon_Transverse.COAD',
 'Adrenal_Gland.ACC',
 'Pancreas.PAAD',
 'Esophagus_Muscularis.ESCA',
 'Stomach.STAD',
 'Thyroid.THCA',
 'Breast_Mammary_Tissue.BRCA',
 'Minor_Salivary_Gland.HNSC',
 'Brain_Anterior_cingulate_cortex_BA24.LGG',
 'Liver.LIHC',
 'Prostate.PRAD',
 'Uterus.UCEC',
 'Lung.LUAD',
 'Lung.LUSC']

In [ ]:
compare_beta=compare_beta[compare_beta["combo"].isin(combos)]

In [ ]:
compare_beta.head()

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)

n=compare_beta["file_name"].unique()
color=iter(cm.rainbow(np.linspace(0,1,len(n))))

for x in n:
    c=next(color)
    canc_filt_dev=compare_beta[compare_beta["file_name"]==x]
    plt.scatter(canc_filt_dev["slope"],canc_filt_dev["BETA"],color=c,marker=".",label=x)


ax.set_xlabel("gtex beta")
ax.plot([ax.get_xlim()[0],ax.get_xlim()[1]], [ax.get_xlim()[0],ax.get_xlim()[1]], ls="--", c="k")
ax.set_ylabel("tcga beta")
plt.legend()
fig.savefig('../plots/gtex.beta.pdf')

## Get gene types of trans associations

In [ ]:
gff3=pd.read_csv("/cellar/users/mpagadal/Data/resources/annotations/gencode.v19.annotation.gff3",delimiter="\t",header=None,comment="#")
gff3.head()

In [ ]:
gff3["ID"]=gff3[8].str.split("gene_id=").str[1].str.split(";").str[0]
gff3["type"]=gff3[8].str.split("gene_type=").str[1].str.split(";").str[0]

In [ ]:
gff3["ID"]=gff3["ID"].str.split(".").str[0]

In [ ]:
mp_type=dict(zip(gff3["ID"],gff3["type"]))

In [ ]:
gtex_trans=gtex_assoc[gtex_assoc["assoc"].isin(trans)]

In [ ]:
gtex_trans["type"]=gtex_trans["gene_id"].map(mp_type)

In [ ]:
gtex_trans=gtex_trans[~(gtex_trans["gtex gene"].isnull())]

In [ ]:
gtex_trans=gtex_trans[~gtex_trans["gtex gene"].isnull()]

In [ ]:
len(gtex_trans["assoc"].unique())

In [ ]:
df_pie=gtex_trans[["assoc","gene_id","type"]].drop_duplicates()["type"].value_counts()

In [ ]:
df_pie=df_pie.reset_index()
df_pie

In [ ]:
f=plt.figure(figsize=(3,3))
labels = df_pie["index"].tolist()
sizes = df_pie["type"].tolist()
plt.pie(sizes,labels=labels,autopct='%1.1f%%')
f.savefig("../plots/pie.type.pdf")